# DIET PROBLEM - PYOMO

*Zuria Bauer Hartwig and David Senén García Hurtado* ( [CAChemE](http://cacheme.org))

Sources:

* Original Problem: [Linear and Integer Programming](https://www.coursera.org/course/linearprogramming) (Coursera Course) - University of Colorado Boulder & University of Colorado System

* Based on the Examples from the Optimization Course = [Taller-Optimizacion-Python-Pyomo](https://github.com/CAChemE/Taller-Optimizacion-Python-Pyomo) from [CAChemE.org](http://cacheme.org/optimizacion-programacion-matematica-con-python-pyomo/)

* [Neos Guide - The Diet Problem](http://www.neos-guide.org/content/diet-problem)

## Big picture

Now let's play with Pyomo. What if we want to maximize the amount of calories we can get for less than $10?

<img src="img/now-talking.jpg" alt="talking" style="width: 600px;"/>

#### 1. Import

In [11]:
from pyomo.environ import *
infinity = float('inf')

# We are importing Pyomo library to our Python enviroment

# Side-note:
# Usually importing all (*) the library is a VERY BAD idea
# By convention Pyomo documentation does that so here we are

# 2. Data

We can use the following data:

- http://fastfoodnutrition.org/mcdonalds/chart
- http://www.fastfoodmenuprices.com/mcdonalds-prices/

In [12]:
# Let's have a look to the data:
# Mac or Linux ('!' meaning to be run through the terminal)
!cat data/foodmax.dat

# Windows via cmd.exe
#!type data\foodmax.dat

# http://fastfoodnutrition.org/mcdonalds/chart
# http://www.fastfoodmenuprices.com/mcdonalds-prices/

#Food + Cost

param:  F:                                                 Cal  :=
"1 Low Fat Milk Jug"                           100    
"Blueberry Pomegranate Smoothie"               220    
"Coca-Cola Classic"                            210    
"Dr Pepper"                                    210    
"Fat Free Chocolate Milk Jug"                  130    
"Hi-C Orange Lavaburst"                        240  
"Mango Pineapple Smoothie"                     220    
"McCafe Classic Lemonade"                      100   
"Minute Maid Apple Juice Box"                  100   
"Minute Maid® Orange Juice"                    190  
"POWERade Mountain Berry Blast"                150  
"Red Flash Extra Small"                        110  
"Red Flash Large"                              300    
"Red Flash Medium"                             220    
"Red Flash Small"                              150     
"

**Note**: Window's users can use `!type data\food-max.dat` command instead of `!cat data/food-max.dat`

### Solving

#### 3. Model

In [13]:
# The AbstractModel class provides a context for defining and initializing abstract optimization models in Pyomo

model = AbstractModel()

#### 4. Sets

In [14]:
## DEFINE SETS
# Set data that is used to define a model instance

# Products
model.F = Set()
# Nutrients
model.N = Set()

#### 5. Parameters

In [15]:
## DEFINE PARAMETERS
# Parameter data that is used to define a model instance

# Calories
model.Cal = Param(model.F, within = PositiveReals)

# Cost
model.a    = Param(model.F, model.N, within = NonNegativeReals)

# Max and Min Cost
model.Cmin = Param(model.N, within = NonNegativeReals, default = 0.0)
model.Cmax = Param(model.N, within = NonNegativeReals, default = infinity)

#### 6. Variables

In [16]:
## VARIABLES
# Decision variables in a model

# Number of servings
model.x = Var(model.F, within = NonNegativeIntegers)


#### 7. Objective

In [17]:
## OBJECTIVE
# Expressions that are minimized or maximized in a model

# MAXIMIZE z(calories)
def calories(model):
    return sum(model.Cal[i] * model.x[i] for i in model.F)
model.calories = Objective(rule=calories, sense=maximize)

#### 8. Constrains

In [18]:
## CONSTRAINS
# Constraint expressions that impose restrictions on variable values in a model

#Max
def cost_max(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return value <= model.Cmax[j]
model.cost_limit_max = Constraint(model.N, rule=cost_max)

#Min
def cost_min(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return model.Cmin[j] <= value 
model.cost_limit_min = Constraint(model.N, rule=cost_min)

#### 9. Solution

In [19]:
##SOLUTION
#Get our Solution:

!pyomo solve --solver=glpk problems/diet-maximize/pyomo-max.py data/foodmax.dat

# windows
#!pyomo solve --solver=glpk problems\diet-maximize\pyomo-max.py data\food-max.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.04] Applying solver
[    0.06] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 3550.0
    Solver results file: results.yml
[    0.06] Applying Pyomo postprocessing actions
[    0.06] Pyomo Finished


#### 10. Results

In [20]:
## Lets have a look to the results now since we are hungry!
# Mac or Linux ('!' meaning to be run through the terminal)
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3550.0
  Upper bound: 3550.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 112
  Number of nonzeros: 223
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 49
      Number of created subproblems: 49
  Error rc: 0
  Time: 0.009647369384765625
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

##### Results

- ** 1 Buffalo Ranch McChicken** 1.00 Dolar
<img src="img/BuffaloRanchMcChicken.png" alt="Hamburger" style="width: 200px;"/>

- ** 6 Sausage Biscuit =** 7.14 Dolar
<img src="img/sausage-biscuit.jpg" alt="Biscuit" style="width: 300px;"/>

- **2 Strawberry & Creme Pie =** 1.78 Dolar
<img src="img/mcdonalds-strawberry-and-creme-pie.jpg" alt="CremePie" style="width: 300px;"/>